<a href="https://colab.research.google.com/github/rklepov/hse-cs-ml-2018-2019/blob/homework/08-spark/HW/hw2/spark_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [*HW2*](https://github.com/rklepov/hse-cs-ml-2018-2019/blob/master/08-spark/02-sql/SparkSQL%20%D0%94%D0%97-2.ipynb)

In [1]:
!pip search spark | grep INSTALLED || pip install pyspark==2.4.0 findspark

  INSTALLED: 2.4.0


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre/"

import findspark
findspark.init('/usr/local/lib/python3.6/dist-packages/pyspark/')

import pyspark

In [0]:
from pyspark.sql import functions
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
sc = pyspark.SparkContext(appName='spark_hw2')
sql = pyspark.HiveContext(sc)

## *Данные*

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
%%bash

cp '/content/gdrive/My Drive/'{zgrab.json,ips,top-1m.csv} ./
ls -hl zgrab.json ips top-1m.csv

-rw------- 1 root root 3.8M Jun 22 06:52 ips
-rw------- 1 root root  22M Jun 22 06:52 top-1m.csv
-rw------- 1 root root 108M Jun 22 06:52 zgrab.json


#### alexa

In [7]:
top_schema = StructType((
    StructField("rank", IntegerType(), nullable=False),
    StructField("site", StringType(), nullable=False),
))

alexa_top_df = sql.read.csv('top-1m.csv', schema=top_schema)

alexa_top_df.show(5)

+----+-------------+
|rank|         site|
+----+-------------+
|   1|   google.com|
|   2|  youtube.com|
|   3| facebook.com|
|   4|    baidu.com|
|   5|wikipedia.org|
+----+-------------+
only showing top 5 rows



#### zgrab

In [8]:
zgrab_content = sql.read.json('zgrab.json').drop('_corrupt_record')

zgrad_codes = (
    zgrab_content
    .select("data.http.response.status_code")
)

zgrad_codes.show(5)

+-----------+
|status_code|
+-----------+
|       null|
|        503|
|        403|
|       null|
|        403|
+-----------+
only showing top 5 rows



#### resolved

In [9]:
resolved_schema = (
    StructType()
    .add("host", "string")
    .add("ip", "string")
)

resolved = sql.read.csv("ips", schema=resolved_schema)

resolved.show(5)

+------------+---------------+
|        host|             ip|
+------------+---------------+
|  google.com| 172.217.17.110|
| youtube.com| 216.58.211.110|
|facebook.com|  157.240.20.35|
|   baidu.com|123.125.114.144|
|   baidu.com| 220.181.38.148|
+------------+---------------+
only showing top 5 rows



## Задание 1

*посчитайте общую статистику: сколько было зафиксировано ответов с каждым статусом*

In [10]:
(
    zgrad_codes
    .groupBy('status_code')
    .count()
    .orderBy(functions.desc('count'))
    .show()
)

+-----------+-----+
|status_code|count|
+-----------+-----+
|       null|36390|
|        200| 7132|
|        404| 1971|
|        403| 1724|
|        503| 1242|
|        400|  658|
|        401|  576|
|        502|  142|
|        405|   35|
|        500|   32|
|        426|   29|
|        302|   23|
|        407|   11|
|        501|    5|
|        410|    2|
|        596|    1|
|        101|    1|
|        504|    1|
|        422|    1|
|        206|    1|
+-----------+-----+



## Задание 2

*Отфильтруйте `zgrab_codes` так чтобы он не содержал пустых значений*

In [11]:
zgrab_codes_nonull = (
    zgrad_codes
    .where(
        ~functions.isnull('status_code')
    )    
)

zgrab_codes_nonull.show(5)

+-----------+
|status_code|
+-----------+
|        503|
|        403|
|        403|
|        200|
|        401|
+-----------+
only showing top 5 rows



## Задание 3

*Посчитайте, сколько ответов сообщали об ошибке*.

- `status_code` *будет принимать значения от 400 до 499 при ошибке на стороне клиента (ошибка 4хх)*

- `status_code` *будет принимать значения от 500 до 599 при ошибке на стороне сервера (ошибка 5хх)*

*Посчитайте, сколько было было ошибок 4хх и 5хх*

In [12]:
(
    zgrab_codes_nonull
    .select(
        functions.substring(
            'status_code', 0, 1
        ).alias('short_code')
    )
    .where(
        functions.col('short_code').isin([4, 5])
    )
    .groupBy('short_code')
    .count()
    .orderBy('count')
    .show()
)

+----------+-----+
|short_code|count|
+----------+-----+
|         5| 1423|
|         4| 5007|
+----------+-----+



## Задание 4

*Определите IP-адреса для сайтов в рейтинге Алексы*

In [13]:
alexa_ips = (
    alexa_top_df
    .join(
        resolved, 
        alexa_top_df.site == resolved.host,
        'left'
    )
)

alexa_ips.show(5)

+----+------------+------------+---------------+
|rank|        site|        host|             ip|
+----+------------+------------+---------------+
|   1|  google.com|  google.com| 172.217.17.110|
|   2| youtube.com| youtube.com| 216.58.211.110|
|   3|facebook.com|facebook.com|  157.240.20.35|
|   4|   baidu.com|   baidu.com| 220.181.38.148|
|   4|   baidu.com|   baidu.com|123.125.114.144|
+----+------------+------------+---------------+
only showing top 5 rows



## Задание 5

*Найдите, какие сайты из рейтинга алексы держат порт 8888 открытым (то есть присутствуют также в датасете zgrab_content и у них status_code – не NULL)*

In [14]:
zgrab_content.show(3)

+--------------------+--------------------+--------------+--------------------+
|                data|               error|            ip|           timestamp|
+--------------------+--------------------+--------------+--------------------+
|               [[,]]|Get http://172.65...|172.65.100.108|2018-05-13T02:30:...|
|[[, [<html style=...|                null|192.230.66.113|2018-05-13T02:30:...|
|[[, [,,, [,,,,,,,...|                null| 159.65.233.69|2018-05-13T02:30:...|
+--------------------+--------------------+--------------+--------------------+
only showing top 3 rows



In [15]:
zgrab_ips = (
    zgrab_content
    .select(
        'data.http.response.status_code',
        functions.col('ip').alias('zg_ip')
    )
    .where(
        ~functions.isnull('status_code')
    )
)

zgrab_ips.show(5)

+-----------+--------------+
|status_code|         zg_ip|
+-----------+--------------+
|        503|192.230.66.113|
|        403| 159.65.233.69|
|        403|185.162.171.15|
|        200|31.201.254.208|
|        401| 151.40.76.245|
+-----------+--------------+
only showing top 5 rows



In [16]:
(
    alexa_ips
    .join(
        zgrab_ips, 
        alexa_ips['ip'] == zgrab_ips['zg_ip'],
        'inner'
    )
    .select(
        'site',
        'status_code',
        'rank',
        'ip'
    )
    .show(15)
)

+--------------------+-----------+-----+---------------+
|                site|status_code| rank|             ip|
+--------------------+-----------+-----+---------------+
|    thedailystar.net|        503| 5976| 107.154.81.121|
|          interac.ca|        503| 7480|  45.60.115.208|
|            comfy.ua|        503|13880|   45.60.72.216|
| nationalexpress.com|        503|17692| 149.126.74.191|
|universal-music.c...|        503|21757|     45.60.71.1|
|        banwagong.hk|        200|26763| 43.251.159.116|
|    surveyjunkie.com|        503|29149|107.154.249.243|
|        tripcase.com|        503|33526|   45.60.46.204|
|            aota.org|        503|34832|   45.60.63.136|
|    translink.com.au|        503|35576|   45.60.15.189|
|         yahui13.com|        403|40241|  198.44.253.56|
|  princehotels.co.jp|        503|40954|   45.60.72.180|
|     lottery-win.org|        503|41344| 107.154.213.27|
|         opal.com.au|        503|42154|  103.28.250.55|
|   inspiredtaste.net|        5

---

[1] [How to upload a file to Google Drive from the command line](https://olivermarshall.net/how-to-upload-a-file-to-google-drive-from-the-command-line/ "How to upload a file to Google Drive from the command line &mdash; OliverMarshall.net")

[2] [Column predicate methods in Spark (isNull, isin, isTrue, isNullOrBlank, etc.)](https://medium.com/@mrpowers/a-tour-of-column-predicate-methods-in-spark-5b1467d53c21 "Column predicate methods in Spark (isNull, isin, isTrue, isNullOrBlank, etc.)")


[3] [Pyspark Joins by Example](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/ "Pyspark Joins by Example &#8211; Learn by Marketing")


[4] [Spark Dataframe distinguish columns with duplicated name](https://stackoverflow.com/questions/33778664/spark-dataframe-distinguish-columns-with-duplicated-name "python - Spark Dataframe distinguish columns with duplicated name - Stack Overflow")